# Data Frame Operations – Basic Transformations such as filtering, aggregations, joins etc

As part of this session we will see basic transformations we can perform on top of Data Frames such as filtering, aggregations, joins etc. We will build end to end application by taking a simple problem statement.

* Data Frame Operations – APIs
* Problem Statement – Get daily product revenue
* Projecting data using select, withColumn and selectExpr
* Filtering data using where or filter
* Joining Data Sets
* Grouping data and performing aggregations
* Sorting data
* Development Life Cycle

### Data Frame Operations – APIs

Let us recap about Data Frame Operations. It is one of the 2 ways we can process Data Frames.

* Selection or Projection – select
* Filtering data – filter or where
* Joins – join (supports outer join as well)
* Aggregations – groupBy and agg with support of functions such as sum, avg, min, max etc
* Sorting – sort or orderBy
* Analytics Functions – aggregations, ranking and windowing functions

# Problem Statement – Get daily product revenue

Here is the problem statement for which we will be exploring Data Frame APIs to come up with final solution.

Get daily product revenue

* orders – order_id, order_date, order_customer_id, order_status
* order_items – order_item_id, order_item_order_id, order_item_product_id, order_item_quantity, order_item_subtotal, order_item_product_price
* Data is comma separated
* We will fetch data using spark.read.csv
* Apply type cast functions to convert fields into their original type where ever is applicable.

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession. \
  builder. \
  master('local'). \
  appName('CSV Example'). \
  getOrCreate()

orders = spark.read. \
  format('csv'). \
  schema('order_id int, order_date string, order_customer_id int, order_status string'). \
  load('/Users/itversity/Research/data/retail_db/orders')

orders.printSchema()
orders.show()

orderItems = spark.read. \
  format('csv'). \
  schema('''order_item_id int, 
            order_item_order_id int, 
            order_item_product_id int, 
            order_item_quantity int,
            order_item_subtotal float,
            order_item_product_price float
         '''). \
  load('/Users/itversity/Research/data/retail_db/order_items')

orderItems.printSchema()
orderItems.show()

AnalysisException: 'Path does not exist: hdfs://nn01.itversity.com:8020/Users/itversity/Research/data/retail_db/orders;'

In [5]:
#  In case you are using pycharm, first you need to create object of type SparkSession
spark = SparkSession. \
  builder. \
  master('local'). \
  appName('CSV Example'). \
  getOrCreate()

ordersCSV = spark.read. \
  csv('/public/retail_db/orders'). \
  toDF('order_id', 'order_date', 'order_customer_id', 'order_status')

orderItemsCSV = spark.read. \
  csv('/public/retail_db/order_items'). \
  toDF('order_item_id', 'order_item_order_id', 'order_item_product_id', 
       'order_item_quantity', 'order_item_subtotal', 'order_item_product_price')

from pyspark.sql.types import IntegerType, FloatType

orders = ordersCSV. \
  withColumn('order_id', ordersCSV.order_id.cast(IntegerType())). \
  withColumn('order_customer_id', ordersCSV.order_customer_id.cast(IntegerType()))

orders.printSchema()
orders.show()

orderItems = orderItemsCSV.\
    withColumn('order_item_id', orderItemsCSV.order_item_id.cast(IntegerType())). \
    withColumn('order_item_order_id', orderItemsCSV.order_item_order_id.cast(IntegerType())). \
    withColumn('order_item_product_id', orderItemsCSV.order_item_product_id.cast(IntegerType())). \
    withColumn('order_item_quantity', orderItemsCSV.order_item_quantity.cast(IntegerType())). \
    withColumn('order_item_subtotal', orderItemsCSV.order_item_subtotal.cast(FloatType())). \
    withColumn('order_item_product_price', orderItemsCSV.order_item_product_price.cast(FloatType()))

orderItems.printSchema()
orderItems.show()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:0

# Projecting data using select, withColumn and selectExpr

Now let us see how we can project data the way we want using select.

* Python classes are dynamic. It means we can change the structure of the class at run time.
* In this case both orders and orderItems are of type DataFrame, but in orders we will be able to access its attributes and in orderItems we will be able to access its attributes (e. g.: orders.order_id and orderItems.order_item_id)
* We can use select and fetch data from the fields we are looking for.
* We can represent data using DataFrame.ColumnName or directly ‘ColumnName’ in select clause – e.g.: <mark> orders.select(orders.order_id, orders.order_date)</mark> and <mark>orders.select('order_id', 'order_date')</mark>
* We can apply necessary functions to manipulate data while it is being projected – <mark>orders.select(substring('order_date', 1, 7)).show()
* We can give aliases to the derived fields using alias function – <mark> orders.select(substring('order_date', 1, 7).alias('order_month')).show()</mark>
* If we want to add new field derived from existing fields we can use withColumn function. First argument is alias and 2nd argument is data processing logic – <mark> orders.withColumn('order_month', substring('order_date', 1, 7).alias('order_month')).show()</mark>

# Filtering data using where or filter

Data Frame have 2 APIs to filter the data, where and filter. They are just synonyms and you can use either of them for filtering.

* You can use filter or where in 2 ways
* One by using class.attributeName and comparing with values – e. g.: <mark>orders.where(orders.order_status == 'COMPLETE').show()</mark>
* Other by passing conditions as literals – e. g.: <mark>orders.where('order_status = "COMPLETE"').show()</mark>
* Make sure both orders and orderItems data frames are created
* Let us see few more examples
 * Get orders which are either COMPLETE or CLOSED
 * Get orders which are either COMPLETE or CLOSED and placed in month of 2013 August
 * Get order items where order_item_subtotal is not equal to product of order_item_quantity and order_item_product_price
 * Get all the orders which are placed on first of every month

In [6]:
# Get orders which are either COMPLETE or CLOSED
orders.where('order_status = "COMPLETE" or order_status = "CLOSED"').show()
orders.where('order_status in ("COMPLETE", "CLOSED")').show()
orders.where((orders.order_status == 'COMPLETE') | (orders.order_status == 'CLOSED')).show()
orders.where((orders.order_status == 'COMPLETE').__or__(orders.order_status == 'CLOSED')).show()
orders.where(orders.order_status.isin('COMPLETE', 'CLOSED')).show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|       1|2013-07-25 00:00:...|            11599|      CLOSED|
|       3|2013-07-25 00:00:...|            12111|    COMPLETE|
|       4|2013-07-25 00:00:...|             8827|      CLOSED|
|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|       6|2013-07-25 00:00:...|             7130|    COMPLETE|
|       7|2013-07-25 00:00:...|             4530|    COMPLETE|
|      12|2013-07-25 00:00:...|             1837|      CLOSED|
|      15|2013-07-25 00:00:...|             2568|    COMPLETE|
|      17|2013-07-25 00:00:...|             2667|    COMPLETE|
|      18|2013-07-25 00:00:...|             1205|      CLOSED|
|      22|2013-07-25 00:00:...|              333|    COMPLETE|
|      24|2013-07-25 00:00:...|            11441|      CLOSED|
|      25|2013-07-25 00:00:...|             9503|      

In [7]:
# Get orders which are either COMPLETE or CLOSED and placed in month of 2013 August

orders.where('order_status in ("COMPLETE", "CLOSED") and order_date like "2013-08%"').show()
orders.where(orders.order_status.isin('COMPLETE', 'CLOSED').__and__(orders.order_date.like('2013-08%'))).show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|    1297|2013-08-01 00:00:...|            11607|    COMPLETE|
|    1298|2013-08-01 00:00:...|             5105|      CLOSED|
|    1299|2013-08-01 00:00:...|             7802|    COMPLETE|
|    1302|2013-08-01 00:00:...|             1695|    COMPLETE|
|    1304|2013-08-01 00:00:...|             2059|    COMPLETE|
|    1305|2013-08-01 00:00:...|             3844|    COMPLETE|
|    1307|2013-08-01 00:00:...|             4474|    COMPLETE|
|    1309|2013-08-01 00:00:...|             2367|      CLOSED|
|    1312|2013-08-01 00:00:...|            12291|    COMPLETE|
|    1314|2013-08-01 00:00:...|            10993|    COMPLETE|
|    1315|2013-08-01 00:00:...|             5660|    COMPLETE|
|    1318|2013-08-01 00:00:...|             4212|    COMPLETE|
|    1319|2013-08-01 00:00:...|             3966|    CO

In [8]:
# Get order items where order_item_subtotal is not equal to product of order_item_quantity and order_item_product_price
orderItems.where('order_item_subtotal != round(order_item_quantity * order_item_product_price, 2)').show()

from pyspark.sql.functions import round
orderItems.where(orderItems.order_item_subtotal != 
                 round((orderItems.order_item_quantity * orderItems.order_item_product_price), 2)
                ).show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+



In [9]:
# Get all the orders which are placed on first of every month
orders.where('date_format(order_date, "dd") = "01"').show()

from pyspark.sql.functions import date_format
orders.where(date_format(orders.order_date, 'dd') == '01').show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|    1297|2013-08-01 00:00:...|            11607|       COMPLETE|
|    1298|2013-08-01 00:00:...|             5105|         CLOSED|
|    1299|2013-08-01 00:00:...|             7802|       COMPLETE|
|    1300|2013-08-01 00:00:...|              553|PENDING_PAYMENT|
|    1301|2013-08-01 00:00:...|             1604|PENDING_PAYMENT|
|    1302|2013-08-01 00:00:...|             1695|       COMPLETE|
|    1303|2013-08-01 00:00:...|             7018|     PROCESSING|
|    1304|2013-08-01 00:00:...|             2059|       COMPLETE|
|    1305|2013-08-01 00:00:...|             3844|       COMPLETE|
|    1306|2013-08-01 00:00:...|            11672|PENDING_PAYMENT|
|    1307|2013-08-01 00:00:...|             4474|       COMPLETE|
|    1308|2013-08-01 00:00:...|            11645|        PENDING|
|    1309|

# Joining Data Sets

Quite often we need to deal with multiple data sets which are related with each other.

* We need to first understand the relationship with respect to data sets
* All our data sets have relationships defined between them.
 * orders and order_items are transaction tables. orders is parent and order_items is child. Relationship is established between the two using order_id (in order_items, it is represented as order_item_order_id)
 * We also have product catalog normalized into 3 tables – products, categories and departments (with relationships established in that order)
 * We also have customers table
 * There is relationship between customers and orders – customers is parent data set as one customer can place multiple orders.
 * There is relationship between product catalog and order_items via products – products is parent data set as one product can be ordered as part of multiple order_items.
* Determine the type of join – inner or outer (left or right or full)
* Data Frames have an API called join to perform joins
* We can make the join outer by passing additional argument
* Let us see few examples
 * Get all the order items corresponding to COMPLETE or CLOSED orders
 * Get all the orders where there are no corresponding order_items
 * Check if there are any order_items where there is no corresponding order in orders data set

In [10]:
# Get all the order items corresponding to COMPLETE or CLOSED orders

orders.where('order_status in ("COMPLETE", "CLOSED")'). \
  join(orderItems, orders.order_id == orderItems.order_item_order_id). \
  show()

+--------+--------------------+-----------------+------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_id|          order_date|order_customer_id|order_status|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+--------+--------------------+-----------------+------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|       1|2013-07-25 00:00:...|            11599|      CLOSED|            1|                  1|                  957|                  1|             299.98|                  299.98|
|       4|2013-07-25 00:00:...|             8827|      CLOSED|            8|                  4|                 1014|                  4|             199.92|                   49.98|
|       4|2013-07-25 00:00:...|             8827|      CLOSED|            7|    

In [11]:
# Get all the orders where there are no corresponding order_items

orders. \
  join(orderItems, orders.order_id == orderItems.order_item_order_id, 'left'). \
  where('order_item_order_id is null'). \
  select('order_id', 'order_date', 'order_customer_id', 'order_status'). \
  show()

orders. \
  join(orderItems, orders.order_id == orderItems.order_item_order_id, 'left'). \
  where(orderItems.order_item_order_id.isNull()). \
  select(orders.order_id, orders.order_date, orders.order_customer_id, orders.order_status). \
  show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|      22|2013-07-25 00:00:...|              333|       COMPLETE|
|      26|2013-07-25 00:00:...|             7562|       COMPLETE|
|      32|2013-07-25 00:00:...|             3960|       COMPLETE|
|      40|2013-07-25 00:00:...|            12092|PENDING_PAYMENT|
|      47|2013-07-25 00:00:...|             8487|PENDING_PAYMENT|
|      53|2013-07-25 00:00:...|             4701|     PROCESSING|
|      54|2013-07-25 00:00:...|            10628|PENDING_PAYMENT|
|      55|2013-07-25 00:00:...|             2052|        PENDING|
|      60|2013-07-25 00:00:...|             8365|PENDING_PAYMENT|
|      76|2013-07-25 00:00:...|             6898|       COMPLETE|
|      78|

In [12]:
# Check if there are any order_items where there is no corresponding order in orders data set

orders. \
  join(orderItems, orders.order_id == orderItems.order_item_order_id, 'right'). \
  where('order_id is null'). \
  select('order_item_id', 'order_item_order_id'). \
  show()

orders. \
  join(orderItems, orders.order_id == orderItems.order_item_order_id, 'right'). \
  where(orders.order_id.isNull()). \
  select(orderItems.order_item_id, orderItems.order_item_order_id). \
  show()

+-------------+-------------------+
|order_item_id|order_item_order_id|
+-------------+-------------------+
+-------------+-------------------+

+-------------+-------------------+
|order_item_id|order_item_order_id|
+-------------+-------------------+
+-------------+-------------------+



# Grouping data and performing aggregations

Many times we want to perform aggregations such as sum, average, minimum, maximum etc with in each group. We need to first group the data and then perform aggregation.

* groupBy is the function which can be used to group the data on one or more columns
* Once data is grouped we can perform all supported aggregations – sum, avg, min, max etc
* We can invoke the functions directly or as part of agg
* agg gives us more flexibility to give aliases to the derived fields
* Let us see few examples
 * Get count by status from orders
 * Get revenue for each order id from order items
 * Get daily product revenue (order_date and order_item_product_id are part of keys, order_item_subtotal is used for aggregation)

In [13]:
from pyspark.sql.functions import count

In [14]:
# Get count by status from orders
# orders.groupBy('order_status').count()
orders.groupBy('order_status'). \
  agg(count('order_status').alias('status_count')). \
  show()

+---------------+------------+
|   order_status|status_count|
+---------------+------------+
|PENDING_PAYMENT|       15030|
|       COMPLETE|       22899|
|        ON_HOLD|        3798|
| PAYMENT_REVIEW|         729|
|     PROCESSING|        8275|
|         CLOSED|        7556|
|SUSPECTED_FRAUD|        1558|
|        PENDING|        7610|
|       CANCELED|        1428|
+---------------+------------+



In [23]:
# Get revenue for each order id from order items 
orderItems.groupBy('order_item_order_id'). \
  sum('order_item_subtotal'). \
  show()

from pyspark.sql.functions import round, sum
orderItems.groupBy('order_item_order_id'). \
  agg(round(sum('order_item_subtotal'), 2).alias('order_revenue')). \
  show()

+-------------------+------------------------+
|order_item_order_id|sum(order_item_subtotal)|
+-------------------+------------------------+
|                148|      479.99000549316406|
|                463|       829.9200096130371|
|                471|      169.98000717163086|
|                496|        441.950008392334|
|               1088|      249.97000885009766|
|               1580|      299.95001220703125|
|               1591|       439.8599967956543|
|               1645|       1509.790023803711|
|               2366|       299.9700012207031|
|               2659|       724.9100151062012|
|               2866|        569.960018157959|
|               3175|      209.97000122070312|
|               3749|      143.97000122070312|
|               3794|      299.95001220703125|
|               3918|       829.9300155639648|
|               3997|       579.9500122070312|
|               4101|      129.99000549316406|
|               4519|        79.9800033569336|
|            

In [24]:
# Get daily product revenue 
# filter for complete and closed orders
# groupBy order_date and order_item_product_id
# Use agg and sum on order_item_subtotal to get revenue

spark.conf.set('spark.sql.shuffle.partitions', '2')

from pyspark.sql.functions import sum, round
orders.where('order_status in ("COMPLETE", "CLOSED")'). \
  join(orderItems, orders.order_id == orderItems.order_item_order_id). \
  groupBy('order_date', 'order_item_product_id'). \
  agg(round(sum('order_item_subtotal'), 2).alias('revenue')). \
  show()

orders.where('order_status in ("COMPLETE", "CLOSED")'). \
  join(orderItems, orders.order_id == orderItems.order_item_order_id). \
  groupBy(orders.order_date, orderItems.order_item_product_id). \
  agg(round(sum(orderItems.order_item_subtotal), 2).alias('revenue')). \
  show()

+--------------------+---------------------+-------+
|          order_date|order_item_product_id|revenue|
+--------------------+---------------------+-------+
|2013-07-25 00:00:...|                  957| 4499.7|
|2013-07-25 00:00:...|                 1014|2798.88|
|2013-07-25 00:00:...|                  365|3359.44|
|2013-07-25 00:00:...|                  926|  79.95|
|2013-07-25 00:00:...|                 1004|5599.72|
|2013-07-25 00:00:...|                  828|  95.97|
|2013-07-25 00:00:...|                   93|  74.97|
|2013-07-25 00:00:...|                  810|  79.96|
|2013-07-25 00:00:...|                  906|  99.96|
|2013-07-25 00:00:...|                  835|  63.98|
|2013-07-26 00:00:...|                  403|3249.75|
|2013-07-26 00:00:...|                  627|3039.24|
|2013-07-26 00:00:...|                  278| 269.94|
|2013-07-26 00:00:...|                  191|6799.32|
|2013-07-26 00:00:...|                 1014|4798.08|
|2013-07-26 00:00:...|                  804| 1

# Sorting data

Now let us see how we can sort the data using sort or orderBy.

* sort or orderBy can be used to sort the data
* We can perform composite sorting by using multiple fields
* By default data will be sorted in ascending order
* We can change the order by using desc function
* Let us see few examples
 * Sort orders by status
 * Sort orders by date and then by status
 * Sort order items by order_item_order_id and order_item_subtotal descending
 * Take daily product revenue data and sort in ascending order by date and then descending order by revenue.

In [25]:
# Sort orders by status
orders.sort('order_status').show()
orders.orderBy('order_status').show()
orders.orderBy(orders.order_status).show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|     527|2013-07-28 00:00:...|             5426|    CANCELED|
|    1435|2013-08-01 00:00:...|             1879|    CANCELED|
|     552|2013-07-28 00:00:...|             1445|    CANCELED|
|     112|2013-07-26 00:00:...|             5375|    CANCELED|
|     564|2013-07-28 00:00:...|             2216|    CANCELED|
|     955|2013-07-30 00:00:...|             8117|    CANCELED|
|    1383|2013-08-01 00:00:...|             1753|    CANCELED|
|     962|2013-07-30 00:00:...|             9492|    CANCELED|
|     607|2013-07-28 00:00:...|             6376|    CANCELED|
|    1013|2013-07-30 00:00:...|             1903|    CANCELED|
|     667|2013-07-28 00:00:...|             4726|    CANCELED|
|    1169|2013-07-31 00:00:...|             3971|    CANCELED|
|     717|2013-07-29 00:00:...|             8208|    CA

In [26]:
#Sort orders by date and then by status
orders.sort('order_date', 'order_status').show()
orders.orderBy(orders.order_date, orders.order_status).show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|      50|2013-07-25 00:00:...|             5225|    CANCELED|
|       1|2013-07-25 00:00:...|            11599|      CLOSED|
|      12|2013-07-25 00:00:...|             1837|      CLOSED|
|       4|2013-07-25 00:00:...|             8827|      CLOSED|
|      37|2013-07-25 00:00:...|             5863|      CLOSED|
|      18|2013-07-25 00:00:...|             1205|      CLOSED|
|      24|2013-07-25 00:00:...|            11441|      CLOSED|
|      25|2013-07-25 00:00:...|             9503|      CLOSED|
|   57754|2013-07-25 00:00:...|             4648|      CLOSED|
|      90|2013-07-25 00:00:...|             9131|      CLOSED|
|      51|2013-07-25 00:00:...|            12271|      CLOSED|
|      57|2013-07-25 00:00:...|             7073|      CLOSED|
|      61|2013-07-25 00:00:...|             4791|      

In [27]:
# Sort order items by order_item_order_id and order_item_subtotal descending
orderItems. \
  sort(orderItems.order_item_order_id, orderItems.order_item_subtotal.desc()). \
  show()

orderItems. \
  orderBy(orderItems.order_item_order_id, orderItems.order_item_subtotal.desc()). \
  show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            6|                  4|                  365|                  5|             299.95|                   59.99|
|            8| 

In [28]:
# Take daily product revenue data and 
# sort in ascending order by date and 
# then descending order by revenue.

spark.conf.set('spark.sql.shuffle.partitions', '2')

from pyspark.sql.functions import sum, round

dailyProductRevenue = orders.where('order_status in ("COMPLETE", "CLOSED")'). \
  join(orderItems, orders.order_id == orderItems.order_item_order_id). \
  groupBy(orders.order_date, orderItems.order_item_product_id). \
  agg(round(sum(orderItems.order_item_subtotal), 2).alias('revenue'))

dailyProductRevenueSorted = dailyProductRevenue. \
  orderBy(dailyProductRevenue.order_date, dailyProductRevenue.revenue.desc())

dailyProductRevenueSorted.show()

+--------------------+---------------------+-------+
|          order_date|order_item_product_id|revenue|
+--------------------+---------------------+-------+
|2013-07-25 00:00:...|                 1004|5599.72|
|2013-07-25 00:00:...|                  191|5099.49|
|2013-07-25 00:00:...|                  957| 4499.7|
|2013-07-25 00:00:...|                  365|3359.44|
|2013-07-25 00:00:...|                 1073|2999.85|
|2013-07-25 00:00:...|                 1014|2798.88|
|2013-07-25 00:00:...|                  403|1949.85|
|2013-07-25 00:00:...|                  502| 1650.0|
|2013-07-25 00:00:...|                  627|1079.73|
|2013-07-25 00:00:...|                  226| 599.99|
|2013-07-25 00:00:...|                   24| 319.96|
|2013-07-25 00:00:...|                  821| 207.96|
|2013-07-25 00:00:...|                  625| 199.99|
|2013-07-25 00:00:...|                  705| 119.99|
|2013-07-25 00:00:...|                  572| 119.97|
|2013-07-25 00:00:...|                  666| 1

# Development Life Cycle

Let us develop the application using Pycharm and run it on the cluster.

* Make sure application.properties have required input path and output path along with execution mode
* Read orders and order_items data into data frames
* Filter for complete and closed orders
* Join with order_items
* Aggregate to get revenue for each order_date and order_item_product_id
* Sort in ascending order by date and then descending order by revenue
* Save the output as CSV format
* Validate using Pycharm
* Ship it to the cluster, run it on the cluster and validate.

[dev]
executionMode = local
input.base.dir = /Users/itversity/Research/data/retail_db
output.base.dir = /Users/itversity/Research/data/bootcamp/pyspark

[prod]
executionMode = yarn-client
input.base.dir = /public/retail_db
output.base.dir = /user/training/bootcamp/pyspark

# Exercises

Try to develop programs for these exercises

* Get number of closed or complete orders placed by each customer
* Get revenue generated by each customer for the month of 2014 January (consider only closed or complete orders)
* Get revenue generated by each product on monthly basis – get product name, month and revenue generated by each product (round off revenue to 2 decimals)
* Get revenue generated by each product category on daily basis – get category name, date and revenue generated by each category (round off revenue to 2 decimals)
* Get the details of the customers who never placed orders